In [1]:
import pandas as pd
import openrouteservice

file_path = "../data/interim/Hauskoordinaten_latlon.csv"
df = pd.read_csv(file_path)

# Initialize your OpenRouteService client (make sure your local ORS instance is running)
client = openrouteservice.Client(base_url='http://localhost:8080/ors')

# Define the starting point coordinates (in lon, lat format)
start_coords = (7.463684770761316, 51.35736663946457)

# Function to calculate distance and duration using OpenRouteService
def calculate_route(row):
    end_coords = (row['lon'], row['lat'])
    try:
        # Request directions
        route = client.directions(coordinates=[start_coords, end_coords], profile='driving-car', format='geojson')
        
        # Extract distance (in km) and duration (in minutes)
        distance_km = route['features'][0]['properties']['segments'][0]['distance'] / 1000  # Convert to km
        duration_min = route['features'][0]['properties']['segments'][0]['duration'] / 60  # Convert to minutes
        
        return pd.Series([distance_km, duration_min])
    except openrouteservice.exceptions.ApiError:
        # Handle any errors that occur during the API request
        return pd.Series([None, None])

# Apply the function to each row to calculate distance and duration
df[['distance_km', 'duration_min']] = df.apply(calculate_route, axis=1)

# Display the updated DataFrame
print(df[['Rechtswert', 'Hochwert', 'lat', 'lon', 'distance_km', 'duration_min']].head())


/tmp/ipykernel_73744/1813169740.py:5: DtypeWarning: Columns (7,9,11,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


     Rechtswert     Hochwert        lat       lon  distance_km  duration_min
0  32396848,472  5681985,307  51.279854  7.521018      14.7579     27.706667
1  32396835,372    5681968,7  51.279703  7.520835      14.7584     27.710000
2  32396889,804  5681986,048  51.279868  7.521610      14.7891     27.831667
3  32396867,586  5681970,397  51.279724  7.521296      14.7821     27.803333
4  32396968,501  5681928,678  51.279367  7.522755      14.9297     28.395000
